### Redis server to support dashapps

## IF YOU WANT TO SEE WARNINGS, COMMENT THIS OUT

In [1]:
import warnings
warnings.filterwarnings("ignore")

### Imports to support redis

In [2]:
import sys
import os

if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))

import pandas as pd

import pyarrow as pa
import redis

import time
import schedule_it



### Imports to support application

In [3]:
import zipfile
import glob
import pandas as pd
import numpy as np

from argparse import ArgumentParser
from argparse import RawDescriptionHelpFormatter
import sys
import os

from barchartacs import build_db
from barchartacs import db_info
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.tools as tls
from plotly.subplots import make_subplots
from plotly.graph_objs.layout import Font,Margin
from IPython import display

import datetime
import pytz
import pathlib
from dateutil.relativedelta import *
import pandas_market_calendars as pmc

import io
from tqdm import tqdm,tqdm_notebook
from barchartacs import pg_pandas as pg
import mibian
import py_vollib
import importlib
from py_vollib import black
from py_vollib.black import implied_volatility
import ipdb
import traceback
import pandas_datareader.data as pdr
from dashapp import dashapp2 as dashapp
import dash_html_components as html
import dash_core_components as dcc


# importlib.reload(build_db)

In [4]:
redis_port = 6379
redis_db = redis.Redis(host = 'localhost',port=6379,db=0)

In [5]:
def update_redis_df(key,df):
    context = pa.default_serialization_context()
    redis_db.set(key, context.serialize(df).to_buffer().to_pybytes())


In [6]:
def schedule_updates(hour,update_callback):
    logger = schedule_it.init_root_logger("logfile.log", "INFO")
    while True:
        logger.info(f"scheduling update for hour {hour}")
        sch = schedule_it.ScheduleNext('hour', hour,logger = logger)
        sch.wait()
        logger.info(f"updating history")
        update_db()
        logger.info(f"sleeping for an hour before next scheduling")
        time.sleep(60*60)
    

### Application cells

In [7]:
opttab = 'sec_schema.options_table'
futtab = 'sec_schema.underlying_table'
pga = db_info.get_db_info()
print(f"futtab max date: {pga.get_sql(f'select max(settle_date) from {futtab}')}")
print(f"opttab max date: {pga.get_sql(f'select max(settle_date) from {opttab}')}")


  sec_db
futtab max date:         max
0  20200710
opttab max date:         max
0  20200710


In [8]:
uk_holidays = open('expiration_data/uk_holidays.csv').readlines()
# uksplit = [','.join([t.strip('\n') for t in l.split(',')]) for l in  uk_holidays]
uksplit = [','.join(l.split(',')) for l in  uk_holidays]
fio = io.StringIO()
fio.writelines(uksplit)
fio.seek(0)
df_ukh = pd.read_csv(fio)
def ukh_to_yyyymmdd(month_day,year):
    md = month_day.strip()
    d = datetime.datetime.strptime(f'{md} {year}', '%B %d %Y')
    return d
#     return d.strftime("%Y-%m-%d")

year_cols = [c for c in df_ukh.columns.values if '20' in str(c)]
for c in year_cols:
    df_ukh[c] = df_ukh[c].apply(lambda s:ukh_to_yyyymmdd(s,c))
uk_holidays = sorted(df_ukh[year_cols].values.reshape(-1))
uk_holidays

[numpy.datetime64('2020-01-02T00:00:00.000000000'),
 numpy.datetime64('2020-04-10T00:00:00.000000000'),
 numpy.datetime64('2020-04-13T00:00:00.000000000'),
 numpy.datetime64('2020-05-04T00:00:00.000000000'),
 numpy.datetime64('2020-05-25T00:00:00.000000000'),
 numpy.datetime64('2020-08-31T00:00:00.000000000'),
 numpy.datetime64('2020-11-30T00:00:00.000000000'),
 numpy.datetime64('2020-12-25T00:00:00.000000000'),
 numpy.datetime64('2020-12-28T00:00:00.000000000'),
 numpy.datetime64('2021-01-01T00:00:00.000000000'),
 numpy.datetime64('2021-04-02T00:00:00.000000000'),
 numpy.datetime64('2021-04-05T00:00:00.000000000'),
 numpy.datetime64('2021-05-03T00:00:00.000000000'),
 numpy.datetime64('2021-05-31T00:00:00.000000000'),
 numpy.datetime64('2021-08-30T00:00:00.000000000'),
 numpy.datetime64('2021-11-30T00:00:00.000000000'),
 numpy.datetime64('2021-12-27T00:00:00.000000000'),
 numpy.datetime64('2021-12-28T00:00:00.000000000'),
 numpy.datetime64('2022-01-03T00:00:00.000000000'),
 numpy.datet

In [9]:
MONTH_CODES = 'FGHJKMNQUVXZ'
DICT_MONTH_CODE = {MONTH_CODES[i]:i+1 for i in range(len(MONTH_CODES))}

from pandas.tseries.holiday import USFederalHolidayCalendar
bday_us = pd.offsets.CustomBusinessDay(calendar=USFederalHolidayCalendar())
# bday_uk = pd.offsets.CustomBusinessDay(calendar=pmc.exchange_calendar_ice.ICEExchangeCalendar().regular_holidays)
bday_uk = pd.offsets.CustomBusinessDay(holidays=uk_holidays)
TIMEZONE = 'US/Eastern'


2020-07-15 07:02:23,304 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.


In [10]:
def get_nth_weekday(year,month,target_weekday,nth_occurrence):
    '''
    weekday is the term that assigns numbers from 0 to 6 to the days of the weeks.
    weekday 0 = monday
    '''
    # get dayofweeks of year,month,1
    weekday_01 = datetime.datetime(year,month,1).weekday()
    if weekday_01 <= target_weekday:
        day_of_month_of_first_occurence = target_weekday - weekday_01
        day_of_month_of_nth_occurence = day_of_month_of_first_occurence + 1 + (nth_occurrence - 1) * 7
    else:
        day_of_month_of_nth_occurence = target_weekday - weekday_01 + 1 + (nth_occurrence) * 7 
    return datetime.datetime(year,month,day_of_month_of_nth_occurence)




def get_ES_options_expiry(symbol):
    '''
    3rd friday of month of symbol
    '''
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    return get_nth_weekday(year,month,4,3)

def get_E6_options_expiry(symbol):
    monthcode_yy = symbol[2:]
    next_month = DICT_MONTH_CODE[monthcode_yy[0]] + 1
    year = 2000 + int(monthcode_yy[1:])
    if next_month>12:
        next_month = 1
        year += 1
    return datetime.datetime(year,next_month,1) - 7*bday_us

def get_CL_options_expiry(symbol):
    '''
    Trading terminates 7 business days before the 26th calendar of the month prior to the contract month.
    '''
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    month = month -1
    if month<1:
        month = 12
        year = year - 1
    return datetime.datetime(year,month,26) - 7*bday_us

def get_NG_options_expiry(symbol):
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    return datetime.datetime(year,month,1) - 4*bday_us

def get_CB_options_expiry(symbol):
    '''
    This is the spec for the CME Brent, but it matches ICE.
    Trading terminates the 4th last London business day of 
    the month, 2 months prior to the contract month 
    except for the February contract month which 
    terminates the 5th last London business day of the 
    month, 2 months prior to the contract month.  
    '''
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    month = month - 1
    if month<1:
        month = 12 + month
        year = year - 1
    days_to_subtract = 4
    if monthcode_yy[0] =='G':
        days_to_subtract = 5
    elif monthcode_yy[0] == 'F':
        days_to_subtract = 3
#     elif monthcode_yy == 'N22':
#         days_to_subtract = 7
    return datetime.datetime(year,month,1,0,0) - days_to_subtract * bday_uk

DICT_PRODUCT = {
    'E6':get_E6_options_expiry,
    'ES':get_ES_options_expiry,
    'CL':get_CL_options_expiry,
    'NG':get_NG_options_expiry,
    'CB':get_CB_options_expiry,
}

    
def get_options_expiry(symbol):
    product = symbol[:2]
    if product not in DICT_PRODUCT:
        return None
    f = DICT_PRODUCT[product]
    return f(symbol)


def dt_from_yyyymmdd(yyyymmdd,hour=0,minute=0,timezone=TIMEZONE):
    y = int(str(yyyymmdd)[0:4])
    m = int(str(yyyymmdd)[4:6])
    d = int(str(yyyymmdd)[6:8])  
    return datetime.datetime(y,m,d,hour,minute,tzinfo=pytz.timezone(timezone))

def yyyymmdd_from_dt(dt):
    y = int(dt.year)
    m = int(dt.month)
    d = int(dt.day)
    return y*100*100 + m*100 + d

def get_dte_pct(trade_yyyymmdd,expiry_yyyymmdd):
    dt_td = dt_from_yyyymmdd(trade_yyyymmdd)
    dt_xp = dt_from_yyyymmdd(expiry_yyyymmdd)
    return ((dt_xp - dt_td).days + 1)/365


In [11]:
def get_postgres_data(contract,pga):
    '''
    get options and futures data for a specific contract from the postgres database
    '''
    osql = f"select * from {opttab} where symbol='{contract}';"
    dfo = pga.get_sql(osql)
    usql = f"select * from {futtab} where symbol='{contract}';"
    dfu = pga.get_sql(usql)
    # Merge options and futures data
    df = dfo.merge(dfu,how='inner',on=['symbol','settle_date'])
    # Get options expiration dates
    df_expiry_dates = dfo[['symbol','settle_date']].groupby('symbol',as_index=False).max()
    calculated_expiry = get_options_expiry(contract)    
    if calculated_expiry is not None:
        expiry_yyyymmdd = yyyymmdd_from_dt(calculated_expiry)
        if expiry_yyyymmdd> df_expiry_dates.iloc[0].settle_date:
            df_expiry_dates.loc[0,'settle_date'] = expiry_yyyymmdd
    return df,df_expiry_dates

In [12]:
USE_PYVOL = True
def lam_pyvol(r):
    try:
        return implied_volatility.implied_volatility(r.close_x,r.close_y,r.strike,.02,r.dte/365, r.pc.lower())
    except:
        return -1
# lam_pyvol = lambda r:implied_volatility.implied_volatility(r.close_x,r.close_y,r.strike,.02,r.dte/365, r.pc.lower())
lam_mibian = lambda r:mibian.BS([r.close_y,r.strike,2,r.dte], callPrice=r.close_x).impliedVolatility

def get_implieds(df,df_expiry_dates):
    df2 = df[['symbol','contract_num','pc','settle_date','strike','close_x','close_y']]
    df9 = df2.copy()
    df9['expiry'] = df_expiry_dates.iloc[0].settle_date
    df9['syear'] = df9.settle_date.astype(str).str.slice(0,4).astype(int)
    df9['smon'] = df9.settle_date.astype(str).str.slice(4,6).astype(int)
    df9['sday'] = df9.settle_date.astype(str).str.slice(6,8).astype(int)
    df9['eyear'] = df9.expiry.astype(str).str.slice(0,4).astype(int)
    df9['emon'] = df9.expiry.astype(str).str.slice(4,6).astype(int)
    df9['eday'] = df9.expiry.astype(str).str.slice(6,8).astype(int)
    df9['sdatetime'] = df9.apply(lambda r:datetime.datetime(r.syear,r.smon,r.sday),axis=1)
    df9['edatetime'] = df9.apply(lambda r:datetime.datetime(r.eyear,r.emon,r.eday),axis=1)
    df9['dte'] = df9.edatetime - df9.sdatetime
    df9.dte = df9.dte.dt.days
    df9 = df9[['symbol','settle_date','pc','contract_num','strike','close_x','close_y','dte']]
    df10 = df9.iloc[:len(df9)].copy()
    df10.index = list(range(len(df10)))
    if USE_PYVOL:
        df10['iv'] = df10.apply(lam_pyvol,axis=1)
    else:
        n = 100
        for i in tqdm_notebook(np.arange(0,len(df10)-n,n)):
                df10.loc[i:i+n,'iv'] = df10.loc[i:i+n].apply(lam_mibian,axis=1)
        print(f'doing remaining {datetime.datetime.now()}')
        i = df10[df10.iv.isna()].index[0]
        df10.loc[i:,'iv'] = df10.loc[i:].apply(lam_mbian,axis=1)
        print(f'done with remaining {datetime.datetime.now()}')
    return df10



In [13]:
def get_contracts(commod):
    all_contracts = pga.get_sql(f"select symbol from {opttab} where substring(symbol,1,2)='{commod}'").symbol.unique()
    return all_contracts


def pseudo_vol(df,df_expiry,fatm=50,rows=100):
    '''
    Return one row for every day, which holds the atm vol for the
    contract whose options data is contained in the DataFrame argument
    df
    
    Get ATM implied vol per day, as if every underlying was the price specified in the arg fatm
    :param df:  DataFrame with columns from get_postgres_data of:
    'symbol','strike','pc','settle_date','close_x','close_y','contract_num'
    :param df_expiry: a one row DataFrame with the expiration date of the options on 
       the symbol whose options data is in df
    :param fatm: the futures atm price to use as the underlying futures for all
      implied vol calculations
    
    '''
    # get desired columns
    df_cl2 = df[['symbol','strike','pc','settle_date','close_x','close_y','contract_num']]
    # find the atm_op: the at the money option (closest to the money)
    df_cl2['atm_op'] = [abs(v) for v in (df_cl2.strike - df_cl2.close_y)]
    df_cl2_atm_op = df_cl2[['settle_date','atm_op']].groupby('settle_date',as_index=False).min()
    df_cl2 = df_cl2_atm_op.merge(df_cl2,on=['settle_date','atm_op'],how='inner')
    df_cl2.index = list(range(len(df_cl2)))
    # get either the put or the call with the same atm_op value and lowest close_x value
    #   close_x is the options price
    df_cl2_put_or_call = df_cl2[['settle_date','close_x']].groupby('settle_date',as_index=False).min()
    df_cl2 = df_cl2_put_or_call.merge(df_cl2,on=['settle_date','close_x'],how='inner')

    # create a new strike
    df_cl2.strike = fatm + df_cl2.strike - df_cl2.close_y
    df_cl2['fprice'] = df_cl2.close_y
    df_cl2.close_y = fatm
    df_cl2 = df_cl2.iloc[-rows:]
    df_cl_fut = df_cl2[['settle_date','fprice']].drop_duplicates()
    df_cl_implieds = get_implieds(df_cl2,df_expiry)
    df_cl_implieds2 = df_cl_implieds[df_cl_implieds.dte>0]
    df_cl_implieds2 = df_cl_implieds2.merge(df_cl_fut,on='settle_date')
    return df_cl_implieds2[['settle_date','iv','fprice']]



In [14]:
def ordered_symbols(sym_list):
    ordered_keys = [
        s[:2]+s[-1]+s[2:4] 
        for s in sorted([v[0:2]+v[-2:]+v[2] for v in sym_list])
    ]
    return ordered_keys
    

In [15]:
def loop_pseudo_vol(dict_contract_df,pga,
                   fatm=50,rows=100):
    ordered_keys = ordered_symbols(list(dict_contract_df.keys()))
    
    dict_df_implied = {}

    for symbol in ordered_keys:
        if dict_contract_df[symbol] is None:
            df,df_expiry = get_postgres_data(symbol,pga)
        else:
            df,df_expiry = dict_contract_df[symbol]
        try:
            df_implied = pseudo_vol(df,df_expiry,fatm=fatm,rows=rows)
            dict_df_implied[symbol] = df_implied
        except Exception as e:
            print(str(e))
            continue
    return dict_df_implied

In [16]:
def update_db():
    commod_list = ['CL','CB']
    monthcodes = [m for m in 'FGHJKMNQUVXZ']
    yyyy_list= range(2011,int(datetime.datetime.now().year)+1)

    dict_contract_df={}
    for commod in tqdm_notebook(commod_list):
        for y in tqdm_notebook(yyyy_list):
            yy = y-2000
            contract_list = [f"{commod}{m}{yy}" for m in monthcodes]
            for symbol in tqdm_notebook(contract_list):
                try:
                    dict_contract_df[symbol] = get_postgres_data(symbol,pga)
                except Exception as e:
                    print(e)
                    continue
    # create constant underlying vol ("pseudo_vol")
    dict_df_implied = loop_pseudo_vol(dict_contract_df,pga) 
    # create a dataframe of dict_df_implied
    df_implied = None
    for symbol in dict_df_implied.keys():
        df_implied_temp = dict_df_implied[symbol]
        df_implied_temp['symbol'] = symbol
        if df_implied is None:
            df_implied = df_implied_temp.copy()
        else:
            df_implied = df_implied.append(df_implied_temp,ignore_index=True)
        df_implied.index = list(range(len(df_implied)))
    update_redis_df('df_cuv_implied',df_implied)    


In [ ]:
if __name__=='__main__':
    update_db()
    schedule_updates(8,update_db)

single positional indexer is out-of-bounds



single positional indexer is out-of-bounds



2020-07-15 13:03:24,537 - root - INFO - scheduling update for hour 8
2020-07-15 13:03:24,541 - root - INFO - Sleeping at time 2020-07-15 13:03:24.541147-04:00 for 18.943333 hours
2020-07-16 17:55:16,781 - root - INFO - Waking at time: 2020-07-16 08:00:00.541147-04:00
2020-07-16 17:55:16,819 - root - INFO - updating history


single positional indexer is out-of-bounds



single positional indexer is out-of-bounds



...

single positional indexer is out-of-bounds



2020-07-26 20:24:54,250 - root - INFO - sleeping for an hour before next scheduling
2020-07-26 21:24:54,211 - root - INFO - scheduling update for hour 8
2020-07-26 21:24:54,226 - root - INFO - Sleeping at time 2020-07-26 21:24:54.221214-04:00 for 10.585000 hours
2020-07-27 15:02:50,084 - root - INFO - Waking at time: 2020-07-27 08:00:00.221214-04:00
2020-07-27 15:02:50,198 - root - INFO - updating history


single positional indexer is out-of-bounds



single positional indexer is out-of-bounds



2020-07-27 15:09:51,373 - root - INFO - sleeping for an hour before next scheduling
2020-07-27 16:09:51,334 - root - INFO - scheduling update for hour 8
2020-07-27 16:09:51,361 - root - INFO - Sleeping at time 2020-07-27 16:09:51.356779-04:00 for 15.835833 hours


## END